# ResNet 50 (Transfer Learning)

In [9]:
import torch
import torchvision.models as models
from torch import nn, optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import pandas as pd
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [10]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load and prepare the data

In [11]:
# Transformations for the dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale-like images to 3 channels for faster computation
    transforms.Resize((256, 256)),               # Resize images to ResNet input size
    transforms.CenterCrop(224),    # Crop the central 224×224 region
    transforms.ToTensor(),                       # Convert images to tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

# Custom dataset class to handle CSV and image folder (To avoid data duplication, but cost more memory)
class SARImageDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = f"{self.image_folder}/{self.data.iloc[idx, 0]}.png"
        label = self.data.iloc[idx, 1]
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image, label

# Load full training dataset
train_val_dataset = SARImageDataset(csv_file="data/train.csv", image_folder="data/images_train", transform=transform)
test_dataset = SARImageDataset(csv_file="data/test.csv", image_folder="data/images_test", transform=transform)

# Split into training and validation datasets
train_size = int(0.8 * len(train_val_dataset))  # 80% for training
val_size = len(train_val_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])


# DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Load pre-trained ResNet50 model and modify it

In [12]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Binary classification: Internal waves (1) or No waves (0)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ishar\.virtualenvs\ML_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Train the model

In [13]:
# Training loop
num_epochs = 10
best_val_loss = float('inf')  # Initialize best validation loss for saving model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "model_outputs_data/best_resnet50_model.pth")

Epoch [1/10], Train Loss: 0.6058, Val Loss: 0.8061, Val Accuracy: 76.80%
Epoch [2/10], Train Loss: 0.4735, Val Loss: 0.4766, Val Accuracy: 78.41%
Epoch [3/10], Train Loss: 0.4329, Val Loss: 0.5873, Val Accuracy: 80.18%
Epoch [4/10], Train Loss: 0.3827, Val Loss: 0.4414, Val Accuracy: 83.55%
Epoch [5/10], Train Loss: 0.3504, Val Loss: 0.3533, Val Accuracy: 85.90%
Epoch [6/10], Train Loss: 0.3136, Val Loss: 0.4314, Val Accuracy: 85.17%
Epoch [7/10], Train Loss: 0.2960, Val Loss: 0.3461, Val Accuracy: 85.02%
Epoch [8/10], Train Loss: 0.2870, Val Loss: 0.3352, Val Accuracy: 86.49%
Epoch [9/10], Train Loss: 0.2728, Val Loss: 0.3196, Val Accuracy: 86.20%
Epoch [10/10], Train Loss: 0.2217, Val Loss: 0.3572, Val Accuracy: 84.73%


### Prediction and Model evaluation

In [14]:
# Test the model
model.load_state_dict(torch.load("model_outputs_data/best_resnet50_model.pth"))  # Load the best model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Compute Evaluation Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='binary')
recall = recall_score(all_labels, all_preds, average='binary')
f1 = f1_score(all_labels, all_preds, average='binary')

print(f"\nTest Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["No Internal Waves", "Internal Waves"]))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

C:\Users\ishar\AppData\Local\Temp\ipykernel_27908\1122781168.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_outputs_data/best_re


Test Accuracy: 86.43%
Precision: 0.92
Recall: 0.80
F1-Score: 0.86

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.82      0.93      0.87       719
   Internal Waves       0.92      0.80      0.86       740

         accuracy                           0.86      1459
        macro avg       0.87      0.87      0.86      1459
     weighted avg       0.87      0.86      0.86      1459


Confusion Matrix:
[[666  53]
 [145 595]]
